# Solving the Knapsack Problem

The Knapsack Problem (KP) is a combinatorial optimization problem which requires the user to select from a range of goods of different values and weights in order to maximize the value of the selected items within a given weight limit. This version is unbounded meaning the we can select items without limit. 

The episodes proceed by selecting items and placing them into the knapsack one at a time until the weight limit is reached or exceeded, at which point the episode ends.

Here, we'll be solving the unbounded version of the problem meaning there is no limit to the number of times we can select any of the items. This version is incredibly small and simply used to confirm the validity of the models and solution mechanisms. We'll be optimizing using two methods, dynamic programming and mathematical programming to demonstrate both methodologies and build confidence in the solutions and environment. From here, we can then expand the environment to larger and more complex versions of the problem involving uncertainty, multiple knapsacks, limited items, etc.

## Knapsack Environment

If you have installed the `or-gym` package and OpenAI gym, you can import the modules and build the environment as follows:

In [1]:
import gym
import or_gym

env = gym.make('Knapsack-v0')

As stated above, this is a very limited version of the problem with only a handful of items, numbered 1 to N. Each of these items has a corresponding weight and a value.

In [2]:
for i in env.item_numbers:
    print('Item\t{}\tWeight\t{}\tValue\t{}'.format(
        i, env.item_weights[i], env.item_values[i]))

Item	0	Weight	1	Value	0
Item	1	Weight	2	Value	1
Item	2	Weight	3	Value	3
Item	3	Weight	6	Value	14
Item	4	Weight	10	Value	20
Item	5	Weight	18	Value	100


The total weight limit for this problem is:

In [4]:
env.max_weight

15

The state is defined as a tuple with three entries: 
- Item weights
- Item values
- Current knapsack weight

This can be accessed with the `_get_obs()` method.

## Solve KP with DP

First, we'll solve this model using a dynamic programming algorithm called **policy iteration**.

Thikning about this carefully, we see that, for the simple dynamic programming approach to the unbounded problem, there are only as many states as the carrying capacity divided by the smallest weight plus one for anything greater than the maximum capacity. In the simple case above, we have a max capacity of 15, and the smallest item we have has a weight of 1. This gives us 16 states. This becomes much more complicated when we have a bounded problem such that the state is defined by the weight and the remaining number of items to pack, so we'll leave that for later.

Policy iteration solves the problem in two steps, first evaluating an initial policy, then updating that policy based on the improved value estimate. The algorithm terminates when there are no longer meaningful changes to the policy.

In [5]:
import numpy as np
from collections import deque

In [13]:
def policy_evaluation(env, policy, gamma):
    # Value iteration
    v = np.zeros(env.max_weight + 1)

    # Terminal state values are 0
    delta = 1e-5
    delta_t = 1
    k = 0
    while delta_t > delta:
        v_new = np.zeros(v.shape)
        for i in range(v_new.shape[0]):
            # Check for terminal state
            if i == env.max_weight:
                continue
            else:
                for action in env.item_numbers:
                    env.current_weight = i
                    env._get_obs()
                    s_1, r, done, _ = env.step(action)
                    prob = policy[i, action]
                    v_new[i] += prob * (r + gamma * v[s_1[-1]])

        k += 1
        delta_t = np.sum(np.abs(v - v_new))
        v = v_new.copy()
    return v

def policy_improvement(env, policy, values, gamma):
    new_policy = np.zeros(policy.shape)
    
    for i in range(new_policy.shape[0]):
        for j in range(new_policy.shape[1]):
            # Get the action values
            action_vals = []
            for action in env.item_numbers:
                # Update state and take step
                env.current_weight = i
                env._get_obs()
                s_1, r, done, _ = env.step(action)
                val = r + gamma * (values[i])
                action_vals.append([val, action])
                
            # Select the best action
            action_vals = np.array(action_vals)
            best_actions = np.argwhere(
                action_vals[:,0]==np.max(action_vals[:, 0])).flatten()
            # Randomize in cases where multiple maximums exist
            if len(best_actions) > 1:
                best_actions = np.random.choice(best_actions)
            new_policy[i, best_actions] = 1

    num_policy_changes = np.sum(policy != new_policy).astype(int)
    return new_policy, num_policy_changes

Additionally, we have a function called `play_policy` that applies the current policy directly to the environment.

In [14]:
def play_policy(env, policy):
    state = env.reset()[-1]
    done = False
    rewards = []
    actions = []
    while done == False:
        action = np.argmax(policy[state])
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        actions.append(action)
        state = next_state[-1]
    return np.array(rewards), np.array(actions)

To begin, we initialize our policy assuming equal probability of all actions. Then we continue to alternate between policy evaluation and improvement until meaningful changes cease.

In [18]:
# Initialize Policy
policy = np.ones((env.max_weight + 1, env.N)) * 1 / env.N
env.reset()
gamma = 1
k = 0
reward_deque = deque(maxlen=3)
action_deque = deque(maxlen=3)
while True:
    values = policy_evaluation(env, policy, gamma)
    policy, num_changes = policy_improvement(env, policy, values, gamma)
    k += 1
    if num_changes == 0:
        break
    # The policy changes cease to matter after a certain point swapping a few
    # high-weight items. We'll evaluate it directly on the environment and
    # and stop if 3 successive policies yield the exact same actions and rewards.
    rewards, actions = play_policy(env, policy)
    reward_deque.append(rewards)
    action_deque.append(actions)
    if len(reward_deque) == 3:
        converge_r = all([np.allclose(reward_deque[i-1], reward_deque[i])
            for i, j in enumerate(reward_deque)])
        converge_a = all([np.allclose(action_deque[i-1], action_deque[i])
            for i, j in enumerate(action_deque)])
        if converge_r and converge_a:
            print("Converged after {} iterations".format(k))
            break

Converged after 3 iterations


The small problem enables the DP solution to converge quickly. The results are given below.

In [22]:
print("Items selected:\t{}\nTotal reward:\t{}".format(actions, rewards.sum()))

Items selected:	[4 2 1]
Total reward:	24


## Mathematical Programming

This model may also be solved via mathematical programming whereby we can construct an integer program according to the constraints given below.

$$\textrm{max} \; z = \sum_i v_i x_i$$

$$\textrm{s.t.} \; \sum_i w_i x_i \leq W$$

$$x_i \in [0, 1]$$

$$v_i, w_i \in \rm I\!R^+$$

- $i$ indexes the items
- $v_i$ is the value of each item $i$
- $w_i$ is the weight of each item $i$
- $x_i$ is a binary decision variable; 1 denotes a selection of the given item for the knapsack.
- $W$ is the weight limit.

We'll optimize this model using Pyomo 5.6 and the GLPK solver.

In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [4]:
# Initialize model
m = ConcreteModel()

# Sets, parameters, and variables
m.W = env.max_weight
m.i = Set(initialize=env.item_numbers)
m.w = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_weights)})
m.v = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_values)})
m.x = Var(m.i, within=NonNegativeIntegers)

# Define contstraints
@m.Constraint(m.i)
def weight_constraint(m, i):
    return sum(m.w[i] * m.x[i] for i in m.i) - m.W <= 0

m.obj = Objective(expr=(
    sum([m.v[i] * m.x[i] for i in m.i])),
    sense=maximize)

In [5]:
# Solve model
solver = SolverFactory('glpk')
results = solver.solve(m, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.57
Parameter(s) specified in the command line:
 --write /tmp/tmp2epjflap.glpk.raw --wglp /tmp/tmp5hasqvq_.glpk.glp --cpxlp
 /tmp/tmpxisnfude.pyomo.lp
Reading problem data from '/tmp/tmpxisnfude.pyomo.lp'...
7 rows, 7 columns, 37 non-zeros
6 integer variables, none of which are binary
84 lines were read
Writing problem data to '/tmp/tmp5hasqvq_.glpk.glp'...
73 lines were written
GLPK Integer Optimizer, v4.57
7 rows, 7 columns, 37 non-zeros
6 integer variables, none of which are binary
Preprocessing...
6 rows, 5 columns, 30 non-zeros
5 integer variables, one of which is binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+01  ratio =  1.000e+01
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 6
Solving LP relaxation...
GLPK Simplex Optimizer, v4.57
6 rows, 5 columns, 30 non-zeros
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (4)
*     3: obj =   3.400000000e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLU

In [6]:
ip_selections = [i for i in m.i if m.x[i]==1]
print("Items selected:\t{}\nTotal reward:\t{}".format(
    ip_selections, m.obj.expr()))

Items selected:	[2]
Total reward:	31.0


As you can see, the selections and the total rewards are identical for both the DP and IP approach.

# UKP Heuristic Solution

In [7]:
def ukp_heuristic(env_name):
    env = gym.make(env_name)
    # Get value-weight ratios
    vw_ratio = env.item_values / env.item_weights
    vw_order = env.item_numbers[np.argsort(vw_ratio)[::-1]]
    actions = []
    rewards = []
    done = False
    while not done:
        max_item = vw_order[0]
        # Check that item fits
        if env.item_weights[max_item] > (env.max_weight - env.current_weight):
            # Remove item from list
            vw_order = vw_order[1:].copy()
            continue
        # Select max_item
        state, reward, done, _ = env.step(max_item)
        actions.append(max_item)
        rewards.append(reward)
        
    return actions, rewards

In [8]:
actions, rewards = ukp_heuristic('Knapsack-v0')

In [10]:
sum(rewards)

31